In [ ]:
import tensorflow as tf
from tensorflow import keras
from google.colab import drive
import numpy as np
import glob
import matplotlib.pyplot as plt

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import cv2
import numpy as np
import random
from tqdm import tqdm
import albumentations as A

image_dir = "/content/gdrive/My Drive/DeepLearning/dataset/images"
mask_dir = "/content/gdrive/My Drive/DeepLearning/dataset/labels_index"

output_image_dir = "/content/gdrive/My Drive/DeepLearning/dataset/aug_images"
output_mask_dir = "/content/gdrive/My Drive/DeepLearning/dataset/aug_masks"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_mask_dir, exist_ok=True)

# Augmentation pipeline basata sul tuo codice ma più ricca e senza flip
transform = A.Compose([
    A.RandomResizedCrop(size=(720, 960), scale=(0.8, 1.0), ratio=(0.9, 1.1), p=0.5),
    A.MotionBlur(blur_limit=5, p=0.5),
    A.Rotate(limit=10, p=0.5),  # rotazioni leggere ±10°
], additional_targets={'mask': 'mask'})

image_filenames = [f for f in os.listdir(image_dir) if f.endswith('.png')]

for filename in tqdm(image_filenames, desc="Augmentation offline"):
    if random.random() > 0.5:
        continue  # skip 50%

    img_path = os.path.join(image_dir, filename)
    mask_path = os.path.join(mask_dir, filename.replace('.png', '.npy'))

    image = cv2.imread(img_path)
    mask_np = np.load(mask_path).astype(np.uint8)
    if len(mask_np.shape) > 2:
        mask_np = np.squeeze(mask_np)

    augmented = transform(image=image, mask=mask_np)
    aug_img = augmented['image']
    aug_mask = augmented['mask']

    base_name = os.path.splitext(filename)[0]
    new_img_name = base_name + "_aug.png"
    new_mask_name = base_name + "_aug.png"

    cv2.imwrite(os.path.join(output_image_dir, new_img_name), aug_img)
    cv2.imwrite(os.path.join(output_mask_dir, new_mask_name), aug_mask)


Augmentation offline: 100%|██████████| 701/701 [04:45<00:00,  2.45it/s]


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

input_mask_dir = "gdrive/My Drive/DeepLearning/dataset/aug_masks"
output_mask_dir = "gdrive/My Drive/DeepLearning/dataset/aug_masks_npy"

os.makedirs(output_mask_dir, exist_ok=True)

mask_files = [f for f in os.listdir(input_mask_dir) if f.endswith('.png')]

for fname in tqdm(mask_files, desc="Convert PNG to NPY"):
    png_path = os.path.join(input_mask_dir, fname)
    mask = cv2.imread(png_path, cv2.IMREAD_UNCHANGED)  # legge valori esatti

    if mask is None:
        print(f"Errore nel leggere: {png_path}")
        continue

    # In caso sia RGB o abbia più canali
    if len(mask.shape) == 3:
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    base_name = os.path.splitext(fname)[0]
    npy_path = os.path.join(output_mask_dir, base_name + ".npy")

    np.save(npy_path, mask.astype(np.uint8))


Convert PNG to NPY: 100%|██████████| 361/361 [00:07<00:00, 49.55it/s]


In [ ]:
import glob
import os

# Directory
image_dirs = [
    'gdrive/My Drive/DeepLearning/dataset/images',
    'gdrive/My Drive/DeepLearning/dataset/aug_images'
]
mask_dirs = [
    'gdrive/My Drive/DeepLearning/dataset/labels_index',
    'gdrive/My Drive/DeepLearning/dataset/aug_masks_npy'
]

# Funzione per trovare la maschera associata a una data immagine
def find_mask_for_image(image_path):
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    for mask_dir in mask_dirs:
        candidate = os.path.join(mask_dir, base_name + '.npy')
        if os.path.exists(candidate):
            return candidate
    raise FileNotFoundError(f"Maschera non trovata per immagine: {image_path}")

# Costruisci tutti i path delle immagini e delle maschere
all_image_paths = []
all_mask_paths = []

for image_dir in image_dirs:
    image_files = sorted(glob.glob(os.path.join(image_dir, '*.png')))
    for img_path in image_files:
        try:
            mask_path = find_mask_for_image(img_path)
            all_image_paths.append(img_path)
            all_mask_paths.append(mask_path)
        except FileNotFoundError as e:
            print(e)  # opzionale: per debug


In [ ]:
orig_images = sorted(glob.glob('gdrive/My Drive/DeepLearning/dataset/images/*.png'))
orig_masks  = sorted(glob.glob('gdrive/My Drive/DeepLearning/dataset/labels_index/*.npy'))

aug_images = sorted(glob.glob('gdrive/My Drive/DeepLearning/dataset/aug_images/*.png'))
aug_masks  = sorted(glob.glob('gdrive/My Drive/DeepLearning/dataset/aug_masks/*.png'))

print("Originali - Immagini:", len(orig_images))
print("Originali - Maschere:", len(orig_masks))
print("Augmentate - Immagini:", len(aug_images))
print("Augmentate - Maschere:", len(aug_masks))

Originali - Immagini: 701
Originali - Maschere: 701
Augmentate - Immagini: 361
Augmentate - Maschere: 361
